In [ ]:
# pip install xlrd

In [ ]:
import sys 
import os
import pandas as pd
import re

In [ ]:
project_path = os.path.abspath('..')
if project_path not in sys.path:
    sys.path.append(project_path)
sys.path

In [ ]:
from config.config import config

In [ ]:
file_path = config
df = pd.read_excel(config['services_excel_file'])
df.shape

In [ ]:
# удаление и переименование

df = df.drop(["Ссылка.Автор.SAMAccount name", "Контрагент"], axis=1)
df.columns = ['link', 'bustype', 'service', 'comment', 'code']
df.shape

In [ ]:
# преобразование в строку, замена переноса строк на пробел, удаление повторяющихся пробелов 

for col in df.columns:
    df[col] = df[col].apply(lambda x: str(x).replace('\n', ' ').strip())
    df[col] = df[col].apply(lambda x: re.sub(r' {2,}', ' ', x).strip())

In [ ]:
# замена "1 в 1"

def replace1in1(row):
    if row['comment'] == '1 в 1':
        return row['service']
    else:
        return row['comment']

df['comment'] = df.apply(replace1in1, axis=1)

In [ ]:
df.describe()

In [ ]:
df.shape

### Удаление транспортных позиций

In [ ]:
def remove_special_characters(text):
    response = re.sub(r'[^\w\s]', ' ', text.lower())
    return re.sub(r'(\s{2,}|\n)', ' ', response)

excluded = ['счета авто грузятся по реестрам', 'услуги грузятся автоматически', 'Счета авто. ']
excluded = [remove_special_characters(x) for x in excluded]
print(excluded)

In [ ]:
df['comment_cleaned'] = df['comment'].apply(remove_special_characters)
df = df[~df['comment_cleaned'].str.strip().isin(excluded)]
df = df.drop('comment_cleaned', axis=1)
df.shape

In [ ]:
df.describe()

### Удаление позиций "услуга русс. яз для акта"

In [ ]:
excluded = ['услуга русс.яз для акта']
excluded = [remove_special_characters(x) for x in excluded]
print(excluded)

In [ ]:
df['comment_cleaned'] = df['comment'].apply(remove_special_characters)
df = df[~df['comment_cleaned'].str.strip().isin(excluded)]
df = df.drop('comment_cleaned', axis=1)
df.shape

### Количество различных комментариев в одной услуге

In [ ]:
uniq_services = df.groupby(['service']).agg(num_comments=('comment', 'count'), uniq_comments=('comment', 'nunique')).reset_index()
uniq_services.tail()

In [ ]:
uniq_services.query("uniq_comments > 1")

### Количество различных услуг в одном комментарии

In [ ]:
uniq_comments = df.groupby(['comment']).agg(num_services=('service', 'count'), uniq_services=('service', 'nunique')).reset_index()
uniq_comments.tail()

In [ ]:
uniq_comments.query("uniq_services > 1")

### Услуги с двумя и более кодами

In [ ]:
service_code_counts = df.groupby('service')['code'].nunique()

services_with_multiple_codes = service_code_counts[service_code_counts > 1]
services_with_multiple_codes

In [ ]:
grouped = df[df['service'].isin(services_with_multiple_codes.index)].groupby(['service', 'code']).size().reset_index(name='count')
display(grouped)

idx = grouped.groupby('service')['count'].idxmin()
min_count_code = grouped.loc[idx]
display(min_count_code)

In [ ]:
df = df[~df['code'].isin(min_count_code.code)]

In [ ]:
df.describe()

================================================================

================================================================

### Группировка

In [ ]:
grouped_comment = df.groupby(['comment']).agg(service_list=('service', lambda x: list(set(x))), codelist = ('code', lambda x: list(set(x)))).reset_index()
grouped_comment['len_service'] = grouped_comment['service_list'].apply(len)
grouped_comment['id'] = list(range(1, grouped_comment.shape[0]+1))

In [ ]:
grouped_comment.shape

In [ ]:
grouped_comment

In [ ]:
grouped_comment[grouped_comment.len_service > 1]

In [ ]:
grouped_comment.to_json(config['uniq_comments_file'], orient='records', force_ascii=False, indent=4)